<a href="https://colab.research.google.com/github/2303a51658/2303a51658-AIML.-/blob/main/LAB%2003_AIML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy

In [ ]:

import random
import math
import functools
from collections import deque

@functools.lru_cache(maxsize=10**6)
def cached_function(x):
    return x * x


class Game:
    def __init__(self):
        self.initial = None

    def actions(self, state):
        """Return a collection of the allowable moves from this state."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def is_terminal(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def utility(self, state, player):
        """Return the value of this final state to the player."""
        raise NotImplementedError


class TicTacToe(Game):
    def __init__(self):

        super().__init__()
        self.initial = [[' ' for _ in range(3)] for _ in range(3)]

    def actions(self, state):
        """Return available moves (empty cells) in the grid."""
        return [(r, c) for r in range(3) for c in range(3) if state[r][c] == ' ']

    def result(self, state, move):
        """Return the resulting state after a move."""
        new_state = [row[:] for row in state]
        player = self.current_player(state)
        new_state[move[0]][move[1]] = player
        return new_state

    def is_terminal(self, state):
        """Check if the game has ended (win or draw)."""
        return self.check_win(state) or len(self.actions(state)) == 0

    def utility(self, state, player):
        """Return 1 if player wins, -1 if player loses, 0 if draw."""
        winner = self.check_win(state)
        if winner == player:
            return 1
        elif winner is not None:
            return -1
        else:
            return 0

    def check_win(self, state):
        """Check if there's a winner on the board."""
        lines = [state[0], state[1], state[2],
                 [state[0][0], state[1][0], state[2][0]],
                 [state[0][1], state[1][1], state[2][1]],
                 [state[0][2], state[1][2], state[2][2]],
                 [state[0][0], state[1][1], state[2][2]],
                 [state[0][2], state[1][1], state[2][0]]]
        for line in lines:
            if line[0] != ' ' and line[0] == line[1] == line[2]:
                return line[0]
        return None

    def current_player(self, state):
        """Determine whose turn it is ('X' or 'O')."""
        num_X = sum(row.count('X') for row in state)
        num_O = sum(row.count('O') for row in state)
        return 'X' if num_X == num_O else 'O'




def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game where strategies are passed for both players."""
    state = game.initial
    while not game.is_terminal(state):
        player = game.current_player(state)
        move = strategies[player](state, game)  # Get the player's move using their strategy
        state = game.result(state, move)
        if verbose:
            print(f"Player {player} made a move: {move}")
            print_board(state)
    return state


def random_strategy(state, game):
    """A simple strategy where the player picks a random move."""
    return random.choice(game.actions(state))


def print_board(state):
    """Helper function to print the Tic-Tac-Toe board."""
    for row in state:
        print(' | '.join(row))
        print('-' * 5)
game = TicTacToe()
strategies = {'X': random_strategy, 'O': random_strategy}


final_state = play_game(game, strategies, verbose=True)

Player X made a move: (1, 1)
  |   |  
-----
  | X |  
-----
  |   |  
-----
Player O made a move: (1, 0)
  |   |  
-----
O | X |  
-----
  |   |  
-----
Player X made a move: (2, 2)
  |   |  
-----
O | X |  
-----
  |   | X
-----
Player O made a move: (2, 0)
  |   |  
-----
O | X |  
-----
O |   | X
-----
Player X made a move: (0, 2)
  |   | X
-----
O | X |  
-----
O |   | X
-----
Player O made a move: (0, 0)
O |   | X
-----
O | X |  
-----
O |   | X
-----


In [ ]:

def alphabeta_search(game, state):
    """Search game to determine best action using alpha-beta pruning."""
    player = game.current_player(state)

    def max_value(state, alpha, beta):
        """Maximizing player (the current player)."""
        if game.is_terminal(state):
            return game.utility(state, player), None
        v = -float('inf')
        best_action = None
        for action in game.actions(state):
            v2, _ = min_value(game.result(state, action), alpha, beta)
            if v2 > v:
                v, best_action = v2, action
            alpha = max(alpha, v)
            if v >= beta:
                break
        return v, best_action

    def min_value(state, alpha, beta):
        """Minimizing opponent."""
        if game.is_terminal(state):
            return game.utility(state, player), None
        v = float('inf')
        best_action = None
        for action in game.actions(state):
            v2, _ = max_value(game.result(state, action), alpha, beta)
            if v2 < v:
                v, best_action = v2, action
            beta = min(beta, v)
            if v <= alpha:
                break  # Alpha cutoff
        return v, best_action


    return max_value(state, -float('inf'), float('inf'))[1]



class Game:
    """Generic Game class."""

    def actions(self, state):
        raise NotImplementedError

    def result(self, state, move):
        raise NotImplementedError

    def is_terminal(self, state):
        raise NotImplementedError

    def utility(self, state, player):
        raise NotImplementedError

    def current_player(self, state):
        raise NotImplementedError


class TicTacToe(Game):
    """A simple Tic-Tac-Toe game implementation."""

    def __init__(self):
        self.initial = [[' ' for _ in range(3)] for _ in range(3)]

    def actions(self, state):
        """Return a list of available moves (empty spaces) on the board."""
        return [(r, c) for r in range(3) for c in range(3) if state[r][c] == ' ']

    def result(self, state, move):
        """Return the state that results from making a move (row, col)."""
        new_state = [row[:] for row in state]
        player = self.current_player(state)
        new_state[move[0]][move[1]] = player
        return new_state

    def is_terminal(self, state):
        """Check if the game has reached a terminal state (win or draw)."""
        return self.check_win(state) or len(self.actions(state)) == 0

    def utility(self, state, player):
        """Return 1 if player wins, -1 if player loses, 0 if draw."""
        winner = self.check_win(state)
        if winner == player:
            return 1
        elif winner is not None:
            return -1
        else:
            return 0

    def check_win(self, state):
        """Check if there's a winner on the board."""
        lines = [state[0], state[1], state[2],
                 [state[0][0], state[1][0], state[2][0]],
                 [state[0][1], state[1][1], state[2][1]],
                 [state[0][2], state[1][2], state[2][2]],
                 [state[0][0], state[1][1], state[2][2]],
                 [state[0][2], state[1][1], state[2][0]]]

        for line in lines:
            if line[0] != ' ' and line[0] == line[1] == line[2]:
                return line[0]
        return None

    def current_player(self, state):
        """Return the current player ('X' or 'O') based on the state."""
        num_X = sum(row.count('X') for row in state)
        num_O = sum(row.count('O') for row in state)
        return 'X' if num_X == num_O else 'O'


def print_board(state):
    """Helper function to print the Tic-Tac-Toe board."""
    for row in state:
        print(' | '.join(row))
        print('-' * 5)




game = TicTacToe()
state = game.initial


state[1][1] = 'X'
print("Initial state:")
print_board(state)


best_move = alphabeta_search(game, state)
print(f"Best move for O: {best_move}")

Initial state:
  |   |  
-----
  | X |  
-----
  |   |  
-----
Best move for O: (0, 0)


In [ ]:

import itertools


class TicTacToe(Game):
    """Play TicTacToe on a height by width board, needing k in a row to win.
       X plays first, followed by O.
    """

    def __init__(self, height=3, width=3, k=3):
        self.height = height
        self.width = width
        self.k = k
        self.squares = [(x, y) for x in range(width) for y in range(height)]
        self.initial = {
            'board': [[' ' for _ in range(width)] for _ in range(height)],
            'to_move': 'X',
            'utility': 0,
        }

    def actions(self, board_state):
        """Return all legal moves on the board."""
        return [(x, y) for x, y in self.squares if board_state['board'][x][y] == ' ']

    def result(self, board_state, move):
        """Return the new game state after a move."""
        new_board = [row[:] for row in board_state['board']]
        player = board_state['to_move']
        new_board[move[0]][move[1]] = player
        next_player = 'O' if player == 'X' else 'X'

        utility = self.compute_utility(new_board, player, move)

        return {
            'board': new_board,
            'to_move': next_player,
            'utility': utility
        }

    def is_terminal(self, board_state):
        """Return True if the board is a terminal state (win or draw)."""
        return board_state['utility'] != 0 or not any(self.actions(board_state))

    def utility(self, board_state, player):
        """Return the value to the player. 1 for win, -1 for loss, 0 otherwise."""
        utility = board_state['utility']
        return utility if player == 'X' else -utility

    def display(self, board_state):
        """Display the board."""
        board = board_state['board']
        for row in board:
            print(' | '.join(row))
            print('-' * (self.width * 2 - 1))

    def compute_utility(self, board, player, move):
        """Check if the player has won with this move."""
        if self.k_in_row(board, player, move):
            return 1 if player == 'X' else -1
        return 0

    def k_in_row(self, board, player, move):
        """Return True if there is a line through (x, y) with k in a row."""
        x, y = move
        directions = [(0, 1), (1, 0), (1, 1), (1, -1)]
        for dx, dy in directions:
            if self.count_in_row(board, player, x, y, dx, dy) >= self.k:
                return True
        return False

    def count_in_row(self, board, player, x, y, dx, dy):
        """Count the number of consecutive pieces by player starting at (x, y) in direction (dx, dy)."""
        total = 1
        for direction in [1, -1]:
            for step in range(1, self.k):
                nx, ny = x + step * dx * direction, y + step * dy * direction
                if 0 <= nx < self.width and 0 <= ny < self.height and board[nx][ny] == player:
                    total += 1
                else:
                    break
        return total




def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game where strategies are passed for both players."""
    state = game.initial
    while not game.is_terminal(state):
        player = state['to_move']
        move = strategies[player](state, game)  # Get the player's move using their strategy
        state = game.result(state, move)
        if verbose:
            print(f"\nPlayer {player} made a move: {move}")
            game.display(state)
    return state


def random_strategy(state, game):
    """A simple strategy where the player picks a random move."""
    return random.choice(game.actions(state))



game = TicTacToe(height=3, width=3, k=3)
strategies = {'X': random_strategy, 'O': random_strategy}


final_state = play_game(game, strategies, verbose=True)


print("\nFinal Board:")
game.display(final_state)

winner = 'X' if final_state['utility'] == 1 else 'O' if final_state['utility'] == -1 else "Draw"
print(f"\nResult: {winner}")


Player X made a move: (1, 2)
  |   |  
-----
  |   | X
-----
  |   |  
-----

Player O made a move: (2, 0)
  |   |  
-----
  |   | X
-----
O |   |  
-----

Player X made a move: (2, 1)
  |   |  
-----
  |   | X
-----
O | X |  
-----

Player O made a move: (2, 2)
  |   |  
-----
  |   | X
-----
O | X | O
-----

Player X made a move: (0, 0)
X |   |  
-----
  |   | X
-----
O | X | O
-----

Player O made a move: (1, 1)
X |   |  
-----
  | O | X
-----
O | X | O
-----

Player X made a move: (1, 0)
X |   |  
-----
X | O | X
-----
O | X | O
-----

Player O made a move: (0, 1)
X | O |  
-----
X | O | X
-----
O | X | O
-----

Player X made a move: (0, 2)
X | O | X
-----
X | O | X
-----
O | X | O
-----

Final Board:
X | O | X
-----
X | O | X
-----
O | X | O
-----

Result: Draw


In [ ]:

import random
from collections import defaultdict

class TicTacToe:
    def __init__(self, height=3, width=3, k=3):
        self.height = height
        self.width = width
        self.k = k
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(width, height, to_move='X', utility=0)

    def actions(self, board):
        """Return a set of legal moves for the current player on the board."""
        return self.squares - set(board.keys())

    def result(self, board, move):
        """Apply a move to the board and return a new board state."""
        player = board.to_move
        new_board = board.new({move: player}, to_move=('O' if player == 'X' else 'X'))
        if self.is_win(new_board, player, move):
            return new_board.new({}, utility=1 if player == 'X' else -1)
        elif not self.actions(new_board):
            return new_board.new({}, utility=0)
        else:
            return new_board

    def utility(self, board, player):
        """Return the utility of the board: 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        """Check if the game has ended."""
        return board.utility != 0 or not self.actions(board)

    def display(self, board):
        """Display the current state of the board."""
        for y in range(self.height):
            print(' | '.join(board.get((x, y), ' ') for x in range(self.width)))
            if y != self.height - 1:
                print('-' * (self.width * 4 - 3))

    def is_win(self, board, player, move):
        """Check if the player has won the game with this move."""
        return any(self.k_in_row(board, player, move, dx, dy)
                   for dx, dy in [(1, 0), (0, 1), (1, 1), (1, -1)])

    def k_in_row(self, board, player, move, dx, dy):
        """Check if there are k marks in a row for player starting from move."""
        x, y = move
        def count(dx, dy):
            return sum(1 for i in range(1, self.k)
                       if board.get((x + i * dx, y + i * dy)) == player)
        return count(dx, dy) + count(-dx, -dy) + 1 >= self.k

In [ ]:

def random_player(game, state):
    """Randomly choose a legal move from the available actions."""
    return random.choice(list(game.actions(state)))

In [ ]:

def minimax_search(state, game):
    """Perform Minimax search to find the best move."""
    player = state.to_move

    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -float('inf'), None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move

    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = float('inf'), None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a
        return v, move

    return max_value(state)[1]

In [ ]:

def play_game(game, strategies, verbose=False):
    """Play a game between two players using the provided strategies."""
    state = game.initial
    while not game.is_terminal(state):
        player = state.to_move
        move = strategies[player](game, state)
        state = game.result(state, move)
        if verbose:
            print(f"Player {player} moves to {move}")
            game.display(state)
            print()
    return state

In [42]:

def alphabeta_search(state, game):
    """Perform Alpha-Beta pruning search to find the best move."""
    player = state.to_move

    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -float('inf'), None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
            if v >= beta:
                return v, move
            alpha = max(alpha, v)
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = float('inf'), None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
            if v <= alpha:
                return v, move
            beta = min(beta, v)
        return v, move

    return max_value(state, -float('inf'), float('inf'))[1]